## Elo Rating method

This is the Elo method.  The example is taken from "Who's #1" by Langville and Meyer and the results match.  Note, the data includes the entire 2009 NFL season, including all playoff games.  

Reference: https://www.gautamnarula.com/rating/

In [614]:
import math 
  
# Function to calculate the Probability 
def Probability(rating1, rating2): 
  
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating1 - rating2) / 1000)) 
  
# Function to calculate Elo rating 
# K is a constant. 
# Player A wins over Player B.  
# tie = true if tie, false otherwise
def EloRating(Ra, Rb, K, tie): 
    
    # To calculate the Winning 
    # Probability of Player B 
    Pb = Probability(Ra, Rb) 
  
    # To calculate the Winning 
    # Probability of Player A 
    Pa = Probability(Rb, Ra) 
  
    # Updating the Elo Ratings 
    if tie:
       Ra = Ra + K * (1/2 - Pa) 
       Rb = Rb + K * (1/2 - Pb) 
    else:        
       Ra = Ra + K * (1 - Pa) 
       Rb = Rb + K * (0 - Pb) 
    
    return Ra, Rb

In [615]:
gameFilename = 'week16wd.csv'
teamFilename = '2018player.txt'

### Load the team names into an array

In [616]:
import pandas as pd

teamNames = pd.read_csv(teamFilename, header = None)
numTeams = len(teamNames)

### Load the games

In [617]:
# columns of games are:
#	column 0 = days since 1/1/0000
#	column 1 = date in YYYYMMDD format
#	column 2 = team1 index
#	column 3 = team1 homefield (1 = home, -1 = away, 0 = neutral)
#	column 4 = team1 score
#	column 5 = team2 index
#	column 6 = team2 homefield (1 = home, -1 = away, 0 = neutral)
#	column 7 = team2 score
games = pd.read_csv(gameFilename, header = None)
games[0].replace(365243, 737300, inplace=True)
numGames = len(games)

### Create Elo ratings

In [618]:
import numpy as np

print(">>>>>>> I'm not including ties.")

eloRatings = np.zeros(numTeams)

for i in range(numGames):
    team1ID = games.loc[i, 2] - 1 # subtracting 1 since python indexes at 0
    team1Score = games.loc[i, 4]
    team2ID = games.loc[i, 5] - 1 # subtracting 1 since python indexes at 0
    team2Score = games.loc[i, 7]

    if team1Score > team2Score:
        team1Rating, team2Rating = EloRating(eloRatings[team1ID], eloRatings[team2ID],32, False)
    elif team1Score < team2Score:
        team2Rating, team1Rating = EloRating(eloRatings[team2ID], eloRatings[team1ID],32, False)
    else:  
        team1Rating, team2Rating = EloRating(eloRatings[team1ID], eloRatings[team2ID],32, True)
        
    eloRatings[team1ID] = team1Rating
    eloRatings[team2ID] = team2Rating

>>>>>>> I'm not including ties.


### Sort and print the ranking of teams

In [619]:
iSort = np.argsort(-eloRatings)

print('\n\n************** ELO Rating Method **************\n')
print('===========================')
print('Rank   Rating      Team   ')
print('===========================')
for i in range(numTeams):
    print(f'{i+1:4d}   {eloRatings[iSort[i]]:.5f}  {teamNames.loc[iSort[i],1]}')

print('')   # extra carriage return



************** ELO Rating Method **************

Rank   Rating      Team   
   1   159.06582   New_Orleans 
   2   120.84036   LA_Rams 
   3   100.45154   Chicago 
   4   99.37179   LA_Chargers 
   5   93.81520   Kansas_City 
   6   71.00275   Houston 
   7   68.97988   New_England 
   8   49.11634   Indianapolis 
   9   48.38967   Dallas 
  10   48.15641   Baltimore 
  11   47.78199   Seattle 
  12   46.25928   Tennessee 
  13   33.80307   Philadelphia 
  14   31.17798   Minnesota 
  15   28.44309   Pittsburgh 
  16   7.73315   Cleveland 
  17   -20.23257   Miami 
  18   -22.13545   Washington
  19   -30.99600   Green_Bay 
  20   -42.37957   Atlanta
  21   -45.25947   Denver 
  22   -48.22676   Cincinnati 
  23   -52.97714   Carolina 
  24   -67.59019   NY_Giants 
  25   -69.72942   Detroit 
  26   -72.15816   Buffalo 
  27   -73.14804   Jacksonville 
  28   -85.60250   Tampa_Bay 
  29   -94.20147   Las_Vegas 
  30   -97.41449   San_Francisco 
  31   -103.61996   NY_Jets 
  32   -12

In [620]:
import pandas as pd

colleyRank = pd.DataFrame(eloRatings)
player = pd.DataFrame(teamNames)
rrank = colleyRank[0]
ewf = player[1]
df = pd.concat([rrank,ewf],axis=1)
dv = df.sort_values(0,ascending=False)
dc = dv.reset_index()
del dc['index']
dc = dc.reset_index()
dc['index'] = dc['index'] + 1
dc = dc. sort_values(1)
dc = dc.rename(columns={'index':'rank'})
dc = dc.rename(columns={1:'teams'})
del dc[0]
dc = dc[['teams','rank']]


In [621]:
de = teamNames.rename(columns={0:'ID', 1:'Name'})
de['ID'].astype(np.int64)
de.set_index('ID')
mydict = dict(zip(de.Name, de.ID))
dc['teams'].replace(mydict, inplace=True)
dc.head()

,teams,rank
31,1,32
19,2,20
9,3,10
25,4,26
22,5,23


In [622]:
def returnRank(t):
    df = dc.loc[dc['teams'] == t]
    return df.iat[0,1]

In [623]:
gamet = pd.read_csv('allgame.csv', header = None)
weekg = gamet.loc[gamet[0] == 17]
numgweek = len(weekg)
correct = []
incorrect = []
num = 1

In [624]:
for i in range(numgweek):
    tea1ID = weekg.iloc[i, 3]
    tea2ID = weekg.iloc[i, 6]
    tea1Score = weekg.iloc[i, 5]
    tea2Score = weekg.iloc[i, 8]
    tea1Rank = returnRank(tea1ID)
    tea2Rank = returnRank(tea2ID)
    if tea1Score > tea2Score and tea1Rank < tea2Rank:
        correct.append(num)
    else:
        incorrect.append(num)

In [625]:
len(correct)/numgweek

0.8125

###### 